### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [74]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, Structure, ARRAY, c_int32
from sys import platform
from bng import bng
import pyproj

In [63]:
if platform == "darwin":
    ext = "dylib"
else:
    ext = "so"

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

In [70]:
# hacky: http://stackoverflow.com/a/30789980/416626
class Int32_2(Structure):
    _fields_ = [("array", ARRAY(c_int32, 2))]

lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)
rust_bng = lib.convert
rust_bng.restype = Int32_2

### Simple test of average conversion speed, Python version

In [67]:
%%timeit -r 10
bng(
    51.44533267, -0.32824866)

10000 loops, best of 10: 31 µs per loop


### Simple test of average conversion speed, Rust 1.0 version

In [71]:
%%timeit -r 10
rust_bng(
    c_float(-0.32824866), c_float(51.44533267))

The slowest run took 499.16 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 10: 2.04 µs per loop


In [2]:
# UK bounding box
N = 55.811741,
E = 1.768960
S = 49.871159, 
W = -6.379880

### A slightly more realistic test, 100k random points within the UK

In [8]:
df = pd.DataFrame({
        'lon': np.random.uniform(W, E, [100000]),
        'lat': np.random.uniform(N, S, [100000])})

In [56]:
%%timeit -r 10
eastings_northings = df.apply(
    lambda x: bng(x['lat'], x['lon']),
    axis=1)

1 loops, best of 10: 6.08 s per loop


In [72]:
%%timeit -r 10
rust_eastings_northings = df.apply(
    lambda x: rust_bng(c_float(x['lon']), c_float(x['lat'])),
    axis=1)

1 loops, best of 10: 3.3 s per loop


### So far: around a 15x improvement on the simple test, around 2x on the "realistic" test

In [77]:
bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

In [78]:
%%timeit -r 10
pyproj.transform(wgs84, bng, -0.32824866, 51.44533267)

The slowest run took 5.49 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 10: 11.8 µs per loop
